<a href="https://colab.research.google.com/github/IzumiSatoshi/Colab-Few-Shot-Patch-Based-Training/blob/main/fspbt_org.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Few-Shot-Patch-Based-Training
Please check latest version in github : https://github.com/IzumiSatoshi/Colab-Few-Shot-Patch-Based-Training/blob/main/Readme.md  
Just running this repo, so advance reading is recommended :  https://github.com/OndrejTexler/Few-Shot-Patch-Based-Training  
And there is cool fork : https://github.com/nicolai256/Few-Shot-Patch-Based-Training
You can probably get better result than this notebook.  

## You need to prepare these 3 dir in project dir  
```
projectName/
  ┝ processName_gen
    └ input_filtered
  └ processName_train
    ├ input_filtered
    └ output
```
**processName_gen/input_filtered** : Put validation images. It is used to monitor the progress of learning.  
**processName_train/input_filtered** : Put original keyframe images.  
**processName_train/output** : Put generated keyframe images by stable diffusion's i2i or something.  

Note that _train/input_filtered image names correspond to _train/output image names. Also, all images should be the same size.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/OndrejTexler/Few-Shot-Patch-Based-Training.git

In [ ]:
# You may see some errors but somehow working in my case
!pip install numpy==1.19.1
!pip install opencv-python==4.4.0.40
!pip install Pillow==7.2.0
!pip install PyYAML==5.3.1
!pip install scikit-image==0.17.2
!pip install scipy==1.5.2
!pip install tensorflow==1.15.3
!pip install torch==1.6.0
!pip install torchvision==0.7.0

In [ ]:
%cd ./Few-Shot-Patch-Based-Training/

In [ ]:
#@markdown images size
height = 512 #@param{type:"number"}
width = 512 #@param{type:"number"}
#@markdown project_dir path and processName
project_dir = "/content/drive/MyDrive/tell_your_world/3" # @param{type:"string"}
process_name = "Miku" # @param{type:"string"}

In [ ]:
# create blank masks whitch is needed for training
import cv2
import numpy as np
import os
train_dir = f"{project_dir}/{process_name}_train"
train_input_filtered_dir = f"{train_dir}/input_filtered"
train_mask_dir = f"{train_dir}/mask"
train_output_dir = f"{train_dir}/output"

gen_dir = f"{project_dir}/{process_name}_gen"
gen_input_filtered_dir = f"{gen_dir}/input_filtered"
gen_mask_dir = f"{gen_dir}/mask"
gen_output_dir = f"{gen_dir}/output"

os.makedirs(train_mask_dir, exist_ok=True)
os.makedirs(gen_mask_dir, exist_ok=True)

blank = np.zeros((height, width, 3))
blank += 255 #white

train_input_filtered_list = os.listdir(train_input_filtered_dir)
for img in train_input_filtered_list:
  mask_img_path = f"{train_mask_dir}/{img}"
  print(mask_img_path)
  cv2.imwrite(mask_img_path, blank)

gen_input_filtered_list = os.listdir(gen_input_filtered_dir)
for img in gen_input_filtered_list:
  mask_img_path = f"{gen_mask_dir}/{img}"
  print(mask_img_path)
  cv2.imwrite(mask_img_path, blank)

## training
It will take at least 30 min.  
Check test result in project/processName_gen/res_p and stop if you satisfied with results.  
Your checkpoints will be saved at project/processName_train/logs_reference_P.

In [ ]:
!python train.py --config "_config/reference_P.yaml" --data_root {train_dir} --log_interval 2000 --log_folder logs_reference_P

## Generate whole video frames

In [ ]:
pth_path = "/content/drive/MyDrive/tell_your_world/3/Miku_train/logs_reference_P/model_00043.pth"#@param{type:"string"}
output_path = f"{gen_dir}/whole_video_output"
!python generate.py --checkpoint {pth_path} --data_root {gen_dir} --dir_input whole_video_input --outdir {output_path} --device "cuda:0"